In [1]:
import os
import random
import time
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

location = 'BJ'
Type = 'O3'

predictors=[
#'longtitude',
# 'latitude',
 'PM2.5_t1',
 'PM10_t1',
 'O3_t1',
 'knn1_PM2.5_t1',
 'knn1_PM10_t1',
 'knn1_O3_t1',
 'knn2_PM2.5_t1',
 'knn2_PM10_t1',
 'knn2_O3_t1',
 'knn3_PM2.5_t1',
 'knn3_PM10_t1',
 'knn3_O3_t1',
 'knn4_PM2.5_t1',
 'knn4_PM10_t1',
 'knn4_O3_t1',
 'hour_sin',
 'hour_cos',
 'week_day_sin',
 'week_day_cos',  
]

#target
target = []
for i in range(6,37):
    target.append(Type+ '_t'+str(i))#'PM2.5_t7'

input_size = len(predictors)
output_size = len(target)
lr = 0.005
per = 0.1
batch_size = 64
time_step = 5

rnn_unit = 30
w1size = 40#fully conective network size 
w2size = 30
w3size = 30

f = open('/home/dedekinds/data_BJ_sample_LSTM.csv')
df = pd.read_csv(f)
df = df.dropna(axis=0,how='any')
#data = df.values
#df = df[:100]#测试

INPUT_DATA = df[predictors].values
OUTPUT_DATA = df[target].values

def get_train_data(batch_size,time_step):
    print('getting data')
    batch_index = []
    #data_train = train_data_split
    #normalized_train_data = (INPUT_DATA[:,:-4]-np.mean(INPUT_DATA[:,:-4],axis=0))/(0.0001+np.std(INPUT_DATA[:,:-4],axis=0))#+0.00001   /0
    #normalized_train_data = np.column_stack((normalized_train_data,INPUT_DATA[:,-4:]))
    train_x = []
    train_y = []
    for i in range(len(INPUT_DATA)-time_step):
        if i % batch_size==0:
            batch_index.append(i)
        x = INPUT_DATA[i:i+time_step ,:input_size]
        x[:,:-4] = (x[:,:-4]-np.min(x[:,:-4],axis=0))/(0.0001+np.max(x[:,:-4],axis=0)-np.min(x[:,:-4],axis=0))
        #y = OUTPUT_DATA[i:i+time_step ,:]#,np.newaxis
        y = OUTPUT_DATA[i:i+time_step ,:]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    batch_index.append(len(INPUT_DATA)-time_step)
    return batch_index,train_x,train_y



weights={
         'in':tf.Variable(tf.random_normal([input_size,rnn_unit])),
         'w1':tf.Variable(tf.random_normal([rnn_unit,w1size])),
         'w2':tf.Variable(tf.random_normal([w1size,w2size])),
#         'w3':tf.Variable(tf.random_normal([w2size,w3size])), 
         'out':tf.Variable(tf.random_normal([w2size,output_size]))
        }
biases={
        'in':tf.Variable(tf.constant(0.1,shape=[rnn_unit,])),
        'w1':tf.Variable(tf.constant(0.1,shape=[w1size,])),
        'w2':tf.Variable(tf.constant(0.1,shape=[w2size,])),
#        'w3':tf.Variable(tf.constant(0.1,shape=[w3size,])), 
        'out':tf.Variable(tf.constant(0.1,shape=[output_size,]))
       }


def lstm(X,keep_prob):     
    batch_size=tf.shape(X)[0]
    time_step=tf.shape(X)[1]
    w_in=weights['in']
    b_in=biases['in']  
    input=tf.reshape(X,[-1,input_size])  #需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    input_rnn=tf.matmul(input,w_in)+b_in
    input_rnn=tf.reshape(input_rnn,[-1,time_step,rnn_unit])  #将tensor转成3维，作为lstm cell的输入
    cell=tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state=cell.zero_state(batch_size,dtype=tf.float32)
    output_rnn,final_states=tf.nn.dynamic_rnn(cell, input_rnn,initial_state=init_state, dtype=tf.float32)  #output_rnn是记录lstm每个输出节点的结果，final_states是最后一个cell的结果
    output=tf.reshape(output_rnn,[-1,rnn_unit]) #作为输出层的输入

    w1 = weights['w1']
    b1 = biases['w1']
    w1_plus_b = tf.matmul(output, w1) + b1
    w1_plus_b = tf.nn.dropout(w1_plus_b, keep_prob)
    w1_output = tf.nn.relu( w1_plus_b, )
    output1=tf.reshape(w1_output,[-1,w1size]) 
    
    w2 = weights['w2']
    b2 = biases['w2']
    w2_plus_b = tf.matmul(output1, w2) + b2
    w2_plus_b = tf.nn.dropout(w2_plus_b, keep_prob)
    w2_output = tf.nn.relu( w2_plus_b, )
    output2=tf.reshape(w2_output,[-1,w2size]) 
    
#    w3 = weights['w3']
#    b3 = biases['w3']
#    w3_plus_b = tf.matmul(output2, w3) + b3
#    w3_plus_b = tf.nn.dropout(w3_plus_b, keep_prob)
#    w3_output = tf.nn.relu( w3_plus_b, )
#    output3=tf.reshape(w3_output,[-1,w3size]) 

    w_out=weights['out']
    b_out=biases['out']
    pred=tf.matmul(output2,w_out)+b_out
    return pred,final_states


def train_lstm(batch_size,time_step):
    print('start training')
    keep_prob = tf.placeholder(tf.float32)
    X=tf.placeholder(tf.float32, shape=[None,time_step,input_size])
    Y=tf.placeholder(tf.float32, shape=[None,time_step,output_size])
    batch_index,train_x,train_y=get_train_data(batch_size,time_step)
    pred,_=lstm(X,keep_prob)
    #损失函数
    loss=tf.reduce_mean(tf.square(tf.reshape(pred,[-1])-tf.reshape(Y, [-1])))
    train_op=tf.train.AdamOptimizer(lr).minimize(loss)
#    saver=tf.train.Saver(tf.global_variables(),max_to_keep=15)
#    module_file = tf.train.latest_checkpoint()    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        saver = tf.train.Saver()
        tf.add_to_collection('X', X)
        tf.add_to_collection('keep_prob', keep_prob)
        tf.add_to_collection('pred', pred)
                
        #重复训练10000次
        for i in range(250):
            for step in range(len(batch_index)-1):
                _,loss_=sess.run([train_op,loss],feed_dict={X:train_x[batch_index[step]:batch_index[step+1]],Y:train_y[batch_index[step]:batch_index[step+1]],keep_prob:0.9})
                #if i % 100 == 0:
            print(i,loss_)
#            if i % 200==0:
#                print("保存模型：",saver.save(sess,'stock2.model',global_step=i))
        model_dir = "/home/dedekinds/LSTM拼接"
        model_name = Type+'_lstm_ann_'+location
        if not os.path.exists(model_dir):
            os.mkdir(model_dir)
        # 保存模型
        saver.save(sess, os.path.join(model_dir, model_name))
        print("保存模型成功！")

        
start_time = time.time()
train_lstm(batch_size,time_step)
print('[{}]: model training time'.format(time.time() - start_time))

/home/dedekinds/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


start training
getting data
0 6113.1274
1 5505.9033
2 5320.888
3 4921.7007
4 4921.966
5 4670.2803
6 4811.752
7 4723.803
8 4639.3306
9 4305.1357
10 4442.062
11 4478.8203
12 4378.7495
13 4285.287
14 4753.6113
15 4575.5205
16 4475.82
17 4263.3813
18 4252.2065
19 4480.1807
20 4233.9243
21 4163.563
22 4582.681
23 4124.109
24 4114.675
25 3943.6553
26 4050.607
27 4016.5586
28 4206.446
29 4160.269
30 4105.2275
31 4055.0535
32 4021.1584
33 4135.488
34 4009.3474
35 3884.135
36 3690.3235
37 4157.014
38 3882.4756
39 3945.607
40 3708.793
41 3757.481
42 3674.5005
43 3815.4211
44 4152.874
45 3835.2751
46 4055.6235
47 3819.5452
48 3792.5852
49 3875.5156
50 3811.5686
51 4047.6294
52 3984.4133
53 3766.2874
54 3943.4492
55 3912.5603
56 3912.8718
57 3801.0698
58 3799.4124
59 3787.2908
60 3743.5232
61 3884.282
62 3751.1882
63 3641.2236
64 3687.0002
65 3913.796
66 3829.536
67 3555.9426
68 3718.5884
69 3861.9285
70 3674.074
71 3886.6511
72 3870.7336
73 3526.6292
74 3738.1863
75 3673.632
76 3751.2878
77 3961.

In [15]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
import math
import tensorflow as tf

duandian=24


time_step = 5
location = 'BJ'
f = open('/home/dedekinds/aqi_data_submission_'+ location +'.csv')#submission获取 hour weekday
ini =pd.read_csv(f)
f = open('/home/dedekinds/'+ location +'_knn_sample_new.csv')#获取knn数据
knn_data = pd.read_csv(f)
f = open('/home/dedekinds/'+'result_'+ location +'_duandian.csv')#获取已经预测的pm25 pm10 o3数据
pre = pd.read_csv(f)





ans = np.zeros((1,3))

for i in [0]:#range(len(ini)):
    #res = []
    ans = np.row_stack((ans,pre[i*duandian:(i+1)*duandian].values[:,1:]))
    
    #本地pm25 pm10 o3
    PRE = pre[i*duandian:(i+1)*duandian].values[-5:,1:]
    
    knn = knn_data.values
    loc = knn[i,4:8]
    
    #knn
    for x in loc:
        PRE = np.column_stack((PRE,pre[x*duandian:(x+1)*duandian].values[-5:,1:]))
    
    #hour
    hour = np.array([duandian-5,duandian-4,duandian-3,duandian-2,duandian-1]).reshape(-1,1)
    PRE = np.column_stack((PRE,np.sin(hour/24*2*math.pi)))
    PRE = np.column_stack((PRE,np.cos(hour/24*2*math.pi)))
    
    #weekday
    week = np.array([(ini[0:1]['week_day'].values[0]+1)%7,
                    (ini[0:1]['week_day'].values[0]+1)%7,
                    (ini[0:1]['week_day'].values[0]+1)%7,
                    (ini[0:1]['week_day'].values[0]+1)%7,
                    (ini[0:1]['week_day'].values[0]+1)%7]).reshape(-1,1)
    PRE = np.column_stack((PRE,np.sin(week/7*2*math.pi)))
    PRE = np.column_stack((PRE,np.cos(week/7*2*math.pi)))
    
    #整理成LSTM适合的数据格式
    PRE[:,:-4] = (PRE[:,:-4]-np.min(PRE[:,:-4],axis=0))/(0.0001+np.max(PRE[:,:-4],axis=0)-np.min(PRE[:,:-4],axis=0))
    
    INPUT = PRE.reshape(1,time_step,len(PRE[0]))
    
    Type = 'PM10'
    model_dir = "/home/dedekinds/LSTM拼接"
    model_name = Type+'_lstm_ann_'+location
    
    sess = tf.Session()
    new_saver = tf.train.import_meta_graph(model_dir+'/'+model_name+'.meta')
    new_saver.restore(sess, model_dir+'/'+model_name)
    X = tf.get_collection('X')[0]
    keep_prob = tf.get_collection('keep_prob')[0]
    pred = tf.get_collection('pred')[0]
    prob=sess.run(pred,feed_dict={X:INPUT,keep_prob:1}) 
    
    
    

INFO:tensorflow:Restoring parameters from /home/dedekinds/LSTM拼接/PM10_lstm_ann_BJ


In [16]:
prob

array([[63.4487  , 62.57241 , 61.889336, 60.969814, 59.65516 , 58.379257,
        57.321877, 55.99231 , 54.67962 , 53.672607, 53.01363 , 52.66938 ,
        52.214027, 51.444794, 50.822456, 50.553635, 50.090088, 49.602013,
        49.130882, 48.74661 , 48.406174, 48.15011 , 48.008904, 48.304558,
        48.595745, 49.238415, 49.6513  , 49.95303 , 50.15399 , 50.663795,
        50.90613 ],
       [40.230446, 41.01702 , 41.785423, 42.401176, 43.030304, 43.716248,
        44.67475 , 45.644363, 46.70635 , 47.836998, 48.948837, 49.957573,
        50.79901 , 51.61229 , 52.295258, 52.886627, 53.449448, 54.00223 ,
        54.546722, 55.029922, 55.5201  , 56.123466, 56.703888, 57.464836,
        58.1009  , 58.778347, 59.35462 , 59.84778 , 60.444572, 61.15737 ,
        61.917862],
       [39.20565 , 39.96733 , 40.72917 , 41.372185, 42.024605, 42.756042,
        43.84909 , 45.021152, 46.29319 , 47.597816, 48.831104, 49.90716 ,
        50.783318, 51.623   , 52.341446, 52.95758 , 53.55945 , 54.169834

In [ ]:
PRE[:,:-4] = (PRE[:,:-4]-np.min(PRE[:,:-4],axis=0))/(0.0001+np.max(PRE[:,:-4],axis=0)-np.min(PRE[:,:-4],axis=0))

In [3]:
import os
import random
import time
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

location = 'BJ'
Type = 'O3'

predictors=[
#'longtitude',
# 'latitude',
 'PM2.5_t1',
 'PM10_t1',
 'O3_t1',
 'knn1_PM2.5_t1',
 'knn1_PM10_t1',
 'knn1_O3_t1',
 'knn2_PM2.5_t1',
 'knn2_PM10_t1',
 'knn2_O3_t1',
 'knn3_PM2.5_t1',
 'knn3_PM10_t1',
 'knn3_O3_t1',
 'knn4_PM2.5_t1',
 'knn4_PM10_t1',
 'knn4_O3_t1',
 'hour_sin',
 'hour_cos',
 'week_day_sin',
 'week_day_cos',  
]

#target
target = []
for i in range(6,37):
    target.append(Type+ '_t'+str(i))#'PM2.5_t7'

input_size = len(predictors)
output_size = len(target)
lr = 0.005
per = 0.1
batch_size = 64
time_step = 5

rnn_unit = 30
w1size = 30#fully conective network size 
w2size = 30
w3size = 10

f = open('/home/dedekinds/data_BJ_sample_LSTM.csv')
df = pd.read_csv(f)
df = df.dropna(axis=0,how='any')
#data = df.values
df = df[:100]#测试

INPUT_DATA = df[predictors].values
OUTPUT_DATA = df[target].values

def get_train_data(batch_size,time_step):
    print('getting data')
    batch_index = []
    #data_train = train_data_split
    #normalized_train_data = (INPUT_DATA[:,:-4]-np.mean(INPUT_DATA[:,:-4],axis=0))/(0.0001+np.std(INPUT_DATA[:,:-4],axis=0))#+0.00001   /0
    #normalized_train_data = np.column_stack((normalized_train_data,INPUT_DATA[:,-4:]))
    train_x = []
    train_y = []
    for i in range(len(INPUT_DATA)-time_step):
        if i % batch_size==0:
            batch_index.append(i)
        x = INPUT_DATA[i:i+time_step ,:input_size]
        x[:,:-4] = (x[:,:-4]-np.min(x[:,:-4],axis=0))/(0.0001+np.max(x[:,:-4],axis=0)-np.min(x[:,:-4],axis=0))
        #y = OUTPUT_DATA[i:i+time_step ,:]#,np.newaxis
        y = OUTPUT_DATA[i ,:]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    batch_index.append(len(INPUT_DATA)-time_step)
    return batch_index,train_x,train_y
batch_index,train_x,train_y = get_train_data(batch_size,time_step)

getting data


In [4]:
np.shape(train_y)

(95, 31)

In [5]:
batch_index

[0, 64, 95]

In [8]:
np.shape(train_y[batch_index[1]:batch_index[2+1]])

IndexError: list index out of range